In [15]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
#Import pandas for reading table from html
import pandas as pd

In [2]:
# Windows users
#update the path to chrome drive
executable_path = {'executable_path': 'c:/Users/covid19/Downloads/chromedriver_win32/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
#With the following line, browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1), 
#we are accomplishing two things.

#One is that we're searching for elements with a specific combination of tag (ul and li) 
#and attribute (item_list and slide, respectively). For example, ul.item_list would be found in HTML as <ul class=”item_list”>.

#Secondly, we're also telling our browser to wait one second before searching for components.
#The optional delay is useful because sometimes dynamic pages take a little while to load, especially if they are image-heavy.

In [5]:
#setup the html parse
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [6]:
#We'll  assign the title and summary text to variables we'll reference later. 
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8882/nasas-perseverance-drives-on-mars-terrain-for-first-time/" target="_self">NASA's Perseverance Drives on Mars' Terrain for First Time</a></div>

In [7]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's Perseverance Drives on Mars' Terrain for First Time"

In [8]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p


'The first trek of the agency’s largest, most advanced rover yet on the Red Planet marks a major milestone before science operations get under way.'

### Featured Images

In [9]:
# Visit URL
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

In [10]:
#Next, we want to click the "Full Image" button. This button will direct our browser to an image slideshow. Let's take a look at the button's HTML tags and attributes with the DevTools.
# 2nd button is the full image butting based on the search in the html code
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1] 
full_image_elem.click()

In [11]:
# Parse the resulting html with soup from the above cell run
html = browser.html
img_soup = soup(html, 'html.parser')

In [12]:
#It's important to note that the value of the src will be different every time the page is updated, 
#so we can't simply record the current value—we would only pull that image each time the code is executed, 
#instead of the most recent one.
#We'll use the image tag and class (<img />and fancybox-img) to build the URL to the full-size image. 
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [14]:
#add the base URL to the image URL from HTML to make an absolute URL
# Use the base URL to create an absolute URL
img_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{img_url_rel}'
img_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

In [16]:
#Tables in HTML are basically made up of many smaller containers. The main container is the <table /> tag.
#Inside the table is <tbody />, which is the body of the table—the headers, columns, and rows.
#<tr /> is the tag for each table row. Within that tag, the table data is
#stored in <td /> tags. This is where the columns are established.
#Instead of scraping each row, or the data in each <td />, 
#we're going to scrape the entire table with Pandas' .read_html() function.

In [17]:
#read table from html
#df = pd.read_html('http://space-facts.com/mars/')[0] With this line,
#we're creating a new DataFrame from the HTML table. 
#The Pandas function read_html() specifically searches for and returns a list of tables found in the HTML.
#By specifying an index of 0, we're telling Pandas to pull only the first table it encounters, or the first item in the list. Then, it turns the table into a DataFrame.
df = pd.read_html('http://space-facts.com/mars/')[0] #read_html read tables from html
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [18]:
# Pandas also has a way to easily convert our DataFrame back into HTML-ready code using the .to_html() function. 

df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [19]:
browser.quit()